# Calculating the PCA for the GMMAT and SMMAT Analyses 
This notebook documents recalculation of PCs for the population specific GMMAT and SMMAT analyses  

Pheno data
 > /mnt/mfs/statgen/alzheimers-family/pheno/pheno_updated_20221121/
 
Geno data: jointly called WGS data from EFIGA and NIALOAD
 > /mnt/mfs/statgen/alzheimers-family/normalized_bed/normalized_merged_autosome.*  


# Common Variants analyses: PCA

In [1]:
# split the geno file per pop. 
ml Singularity
for i in African European Hispanic; do
sos run ~/project2022/notebook/AD/xqtl-pipeline/pipeline/GWAS_QC.ipynb qc:1 \
  --cwd /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA \
  --genoFile /mnt/mfs/statgen/alzheimers-family/normalized_bed/normalized_merged_autosome.bed \
  --keep_samples /mnt/mfs/statgen/alzheimers-family/pheno/pheno_updated_20221121/$i.id \
  --name $i \
  --container /mnt/vast/hpc/csg/containers/lmm.sif
done

INFO: Running basic QC filters: Filter SNPs and select individuals
INFO: basic QC filters is completed.
INFO: basic QC filters output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.African.filtered.bed
INFO: Workflow qc (ID=w40505755e477e8fe) is executed successfully with 1 completed step.
INFO: Running basic QC filters: Filter SNPs and select individuals
INFO: basic QC filters is completed.
INFO: basic QC filters output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.European.filtered.bed
INFO: Workflow qc (ID=w806094c862aae420) is executed successfully with 1 completed step.
INFO: Running basic QC filters: Filter SNPs and select individuals
INFO: basic QC filters is completed.
INFO: basic QC filters output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.Hispanic.filtered.bed
INFO: Workflow qc (ID=wd71adae04de24e26) is executed successfully with 1 completed step.


In [2]:
for i in African European Hispanic; do
sos run ~/project2022/notebook/AD/xqtl-pipeline/pipeline/GWAS_QC.ipynb king \
  --cwd /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/King \
  --container /mnt/vast/hpc/csg/containers/lmm.sif \
  --genoFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.$i.filtered.bed
done

INFO: Running king_1: Inference of relationships in the sample to identify closely related individuals
INFO: king_1 is completed.
INFO: king_1 output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/King/normalized_merged_autosome.African.filtered.kin0
INFO: Running king_2: Select a list of unrelated individual with an attempt to maximize the unrelated individuals selected from the data
INFO: king_2 is completed.
INFO: king_2 output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/King/normalized_merged_autosome.African.filtered.related_id
INFO: Running king_3: Split genotype data into related and unrelated samples, if related individuals are detected
INFO: king_3 is completed.
INFO: king_3 output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/King/normalized_merged_autosome.African.filtered.unrelated.bed /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/King/normalized_merged_autosome.African.filtered.related.bed
INFO: Workflow king (ID=w1

In [9]:
for i in African European Hispanic; do
# unrelated individuals
ml Singularity
sos run ~/project2022/notebook/AD/xqtl-pipeline/pipeline/GWAS_QC.ipynb qc \
    --cwd /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA \
    --genoFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.$i.filtered.bed \
    --remove_samples /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/King/normalized_merged_autosome.$i.filtered.related_id \
    --maf_filter 0.01 \
    --geno_filter 0.1 \
    --mind_filter 0.1 \
    --hwe_filter 5e-08 \
    --name unrelated \
    --window 50 \
    --shift 10 \
    --r2 0.2 \
    --container /mnt/mfs/statgen/containers/lmm.sif
done

INFO: Running basic QC filters: Filter SNPs and select individuals
INFO: qc_1 (index=0) is ignored due to saved signature
INFO: basic QC filters output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.African.filtered.unrelated.filtered.bed
INFO: Running LD pruning: LD prunning and remove related individuals (both ind of a pair)
INFO: LD pruning is completed.
INFO: LD pruning output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.African.filtered.unrelated.filtered.prune.bed /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.African.filtered.unrelated.filtered.prune.in
INFO: Workflow qc (ID=wb8444235f09d5938) is executed successfully with 1 completed step and 1 ignored step.
INFO: Running basic QC filters: Filter SNPs and select individuals
INFO: qc_1 (index=0) is ignored due to saved signature
INFO: basic QC filters output:   /mnt/mfs/statgen/alzheimers-family/AD_common_var

In [10]:
cat /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.African.filtered.unrelated.filtered.prune.in | wc -l
cat /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.African.filtered.unrelated.filtered.prune.out | wc -l

3799108
16137551


In [11]:
cat /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.African.filtered.unrelated.filtered.log | grep 'samples ('
cat /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.African.filtered.unrelated.filtered.prune.log | grep 'remain'

45 samples (0 females, 0 males, 45 ambiguous; 45 founders) loaded from
--extract: 3812519 variants remaining.
3812519 variants remaining after main filters.


In [12]:
wc -l /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.African.filtered.unrelated.filtered.prune.bim

3812519 /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.African.filtered.unrelated.filtered.prune.bim


In [15]:
for i in African European Hispanic; do
# related individuals same set of variants
sos run ~/project2022/notebook/AD/xqtl-pipeline/pipeline/GWAS_QC.ipynb qc:1 \
    --cwd /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA \
    --genoFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.$i.filtered.bed \
    --keep_samples /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/King/normalized_merged_autosome.$i.filtered.related_id\
    --keep_variants /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.$i.filtered.unrelated.filtered.prune.in \
    --maf_filter 0 \
    --geno_filter 1 \
    --mind_filter 0.1 \
    --hwe_filter 0 \
    --name related \
    --container /mnt/mfs/statgen/containers/lmm.sif
done

INFO: Running basic QC filters: Filter SNPs and select individuals
INFO: basic QC filters is completed.
INFO: basic QC filters output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.African.filtered.related.filtered.extracted.bed
INFO: Workflow qc (ID=w91cfa771347598a6) is executed successfully with 1 completed step.
INFO: Running basic QC filters: Filter SNPs and select individuals
INFO: basic QC filters is completed.
INFO: basic QC filters output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.European.filtered.related.filtered.extracted.bed
INFO: Workflow qc (ID=w83ef60972346f2b2) is executed successfully with 1 completed step.
INFO: Running basic QC filters: Filter SNPs and select individuals
INFO: basic QC filters is completed.
INFO: basic QC filters output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.Hispanic.filtered.related.filtered.extracted.bed
INFO: Work

In [16]:
cat /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.African.filtered.related.filtered.extracted.log | grep 'remain'
cat /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.African.filtered.unrelated.filtered.log | grep 'samples ('

--extract: 3812887 variants remaining.
--keep: 68 samples remaining.
68 samples (0 females, 0 males, 68 ambiguous; 68 founders) remaining after main
3812887 variants remaining after main filters.
45 samples (0 females, 0 males, 45 ambiguous; 45 founders) loaded from


In [17]:
for i in African European Hispanic; do
sos run ~/project2022/notebook/AD/xqtl-pipeline/code/data_preprocessing/genotype/PCA.ipynb flashpca \
    --cwd /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots\
    --genoFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.$i.filtered.unrelated.filtered.prune.bed \
    --phenoFile /mnt/mfs/statgen/alzheimers-family/pheno/pheno_updated_20221121/$i.txt \
    --k 10 \
    --label_col pop \
    --pop_col pop \
    --container /mnt/mfs/statgen/containers/lmm.sif
done

INFO: Running flashpca_1: Run PCA analysis using flashpca
INFO: flashpca_1 is completed.
INFO: flashpca_1 output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/African.pca.rds
INFO: Running flashpca_2: 
INFO: flashpca_2 is completed (pending nested workflow).
INFO: Running detect_outliers: Calculate Mahalanobis distance per population and report outliers
INFO: detect_outliers is completed.
INFO: detect_outliers output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/African.pca.mahalanobis.rds /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/African.pca.outliers... (5 items)
INFO: flashpca_2 output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/African.pca.mahalanobis.rds /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/African.pca.outliers... (5 items)
INFO: Running flashpca_3: 
INFO: flashpca_3 is completed (pending nested workflow).
INFO: Running plot_pca: Plot PCA results. Can be used independen

In [ ]:
# step 5 project back for related samples, detect and generate outlier samples
for i in African European Hispanic; do
sos run ~/project2022/notebook/AD/xqtl-pipeline/code/data_preprocessing/genotype/PCA.ipynb project_samples \
    --cwd /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots \
    --genoFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.$i.filtered.related.filtered.extracted.bed \
    --phenoFile /mnt/mfs/statgen/alzheimers-family/pheno/pheno_updated_20221121/$i.txt\
    --pca_model /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/$i.pca.rds \
    --k 10 \
    --label_col pop \
    --pop_col pop \
    --container /mnt/vast/hpc/csg/containers/flashpcaR.sif
done

INFO: Running project_samples_1: Project back to PCA model additional samples
INFO: project_samples_1 is completed.
INFO: project_samples_1 output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/African.pca.projected.rds
INFO: Running project_samples_2: 
INFO: project_samples_2 is completed (pending nested workflow).
INFO: Running detect_outliers: Calculate Mahalanobis distance per population and report outliers
INFO: detect_outliers is completed.
INFO: detect_outliers output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/African.pca.projected.mahalanobis.rds /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/African.pca.projected.outliers... (5 items)
INFO: project_samples_2 output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/African.pca.projected.mahalanobis.rds /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/African.pca.projected.outliers... (5 items)
INFO: Running project_samples_3: 
INFO: pro

In [ ]:
# step 5 part 2
for i in African European Hispanic; do
sos run ~/project2022/notebook/AD/xqtl-pipeline/code/data_preprocessing/genotype/PCA.ipynb plot_pca \
    --cwd /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots \
    --genoFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.$i.filtered.bed \
    --phenoFile /mnt/mfs/statgen/alzheimers-family/pheno/pheno_updated_20221121/$i.txt \
    --pop_col pop \
    --label_col pop\
    --k 10 \
    --plot_data /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/$i.pca.projected.rds\
    --container /mnt/vast/hpc/csg/containers/flashpcaR.sif
done

INFO: Running plot_pca: Plot PCA results. Can be used independently as "plot_pca" or combined with other workflow as eg "flashpca+plot_pca"
INFO: plot_pca is completed.
INFO: plot_pca output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/African.pca.projected.pc.png /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/African.pca.projected.scree.png... (3 items)
INFO: Workflow plot_pca (ID=w11ec71d77a952316) is executed successfully with 1 completed step.
INFO: Running plot_pca: Plot PCA results. Can be used independently as "plot_pca" or combined with other workflow as eg "flashpca+plot_pca"
INFO: plot_pca is completed.
INFO: plot_pca output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/European.pca.projected.pc.png /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/plots/European.pca.projected.scree.png... (3 items)
INFO: Workflow plot_pca (ID=w107043c27168f420) is executed successfully with 1 completed step.
INFO: Running pl

# Gnerate QCed genoFile without LD pruning to use in the GMMAT analysis


In [3]:
for i in African European Hispanic; do
# unrelated individuals
sos run ~/project2022/notebook/AD/xqtl-pipeline/pipeline/GWAS_QC.ipynb qc_no_prune \
    --cwd /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/ \
    --genoFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.$i.filtered.bed \
    --remove_samples /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/King/normalized_merged_autosome.$i.filtered.related_id \
    --maf_filter 0.01 \
    --geno_filter 0.1 \
    --mind_filter 0.1 \
    --hwe_filter 5e-08 \
    --name unrelated \
    --container /mnt/mfs/statgen/containers/lmm.sif
# related individuals same set of variants
sos run ~/project2022/notebook/AD/xqtl-pipeline/pipeline/GWAS_QC.ipynb qc:1 \
    --cwd /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/ \
    --genoFile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/normalized_merged_autosome.$i.filtered.bed \
    --keep_samples /mnt/mfs/statgen/alzheimers-family/AD_common_variants/PCA/King/normalized_merged_autosome.$i.filtered.related_id \
    --keep_variants /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/normalized_merged_autosome.$i.filtered.unrelated.filtered.bim \
    --maf_filter 0 \
    --geno_filter 0.1 \
    --mind_filter 0.1 \
    --hwe_filter 0 \
    --name related \
    --container /mnt/mfs/statgen/containers/lmm.sif 
done

INFO: Running qc_no_prune: Filter SNPs and select individuals
INFO: qc_no_prune is completed.
INFO: qc_no_prune output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/normalized_merged_autosome.African.filtered.unrelated.filtered.bed
INFO: Workflow qc_no_prune (ID=w214389013524cfa6) is executed successfully with 1 completed step.
INFO: Running basic QC filters: Filter SNPs and select individuals
INFO: basic QC filters is completed.
INFO: basic QC filters output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/normalized_merged_autosome.African.filtered.related.filtered.extracted.bed
INFO: Workflow qc (ID=w55663afe177e45bb) is executed successfully with 1 completed step.
INFO: Running qc_no_prune: Filter SNPs and select individuals
INFO: qc_no_prune is completed.
INFO: qc_no_prune output:   /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/normalized_merged_autosome.European.filtered.unrelated.filtered.bed
INFO: Workflow qc_no_prune (ID

In [4]:
# merge two data-sets
bash: container= '/mnt/mfs/statgen/containers/lmm.sif'
for i in African European Hispanic; do
    plink --bfile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/normalized_merged_autosome.$i.filtered.related.filtered.extracted \
         --bmerge /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/normalized_merged_autosome.$i.filtered.unrelated.filtered.bed \
                  /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/normalized_merged_autosome.$i.filtered.unrelated.filtered.bim \
                  /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/normalized_merged_autosome.$i.filtered.unrelated.filtered.fam \
        --make-bed --keep-allele-order --out /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/$i
done

bash: bash:: command not found
PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/African.log.
Options in effect:
  --bfile /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/normalized_merged_autosome.African.filtered.related.filtered.extracted
  --bmerge /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/normalized_merged_autosome.African.filtered.unrelated.filtered.bed /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/normalized_merged_autosome.African.filtered.unrelated.filtered.bim /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/normalized_merged_autosome.African.filtered.unrelated.filtered.fam
  --keep-allele-order
  --make-bed
  --out /mnt/mfs/statgen/alzheimers-family/AD_common_variants/geno_qced/African

515677 MB RAM detected; reserving 25783